## Cálculo de predicciones de clasificacion finales de ligas

Imports

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy.optimize as optim

Función para la obtención de los datos

In [15]:
#Devuelve el índice en la lista del nombre del equipo introducido
def getindexteam(team, classification):
  i = 0
  for t in classification:
    if t[0] == team:
      return i
    else:
      i = i+1

def getdatafromcsv(filename):

  df = pd.DataFrame(columns =['Home', 'Away', 'pointsH', 'pointsA',
                            'xglocal','xgaway','xglocalconceded','xgawayconceded',
                            'localstreak','awaystreak', 'especial',
                            'mgh','mga','mah','maa','persavH','persavA','Result'])

  #archive = files.upload()
  datacsv = pd.read_csv(filename, sep=";")

  #columnas:  
  #0 nombre,  1 puntos en clasificacion,        2 partidos jugados como local,  3 partidos jugados como visitante, 
  #           4 xg a favor como local,          5 xg en contra como local,      6 xg a favor como visitante,     
  #           7 xg en contra como visitante     8 porcentaje paradas portero

  classification = []

  for i in range(10):
    classification.append([datacsv.loc[i][0], 0, 0, 0, 0, 0, 0, 0, 0])
    classification.append([datacsv.loc[i][1], 0, 0, 0, 0, 0, 0, 0, 0])

  a = []

  for i in range(len(datacsv)):

    localteam = datacsv.loc[i][0] #nombre equipo lcoal
    awayteam = datacsv.loc[i][1] #nombre equipo visitante
    indexlocalteam = getindexteam(localteam, classification)
    indexawayteam = getindexteam(awayteam, classification)

    pointsH = classification[indexlocalteam][1] #puntos clasificacion local
    pointsA = classification[indexawayteam][1] #puntos clasificacion visitante

    rangelocal = 0
    localstreak = 0
    aux = df[df.Home == localteam]
    aux1 = df[df.Away == localteam]
    aux = pd.concat([aux,aux1])
    aux = aux.sort_index()
    aux = aux.iloc[::-1]
    
    if len(aux) >= 5:
      rangelocal = 5
    else:
      rangelocal = len(aux)
    for t in range(rangelocal):
      if aux.iloc[t,0] == localteam:
        if aux.iloc[t,17] == "H":
          localstreak += 3
        elif aux.iloc[t,17] == "D":
          localstreak += 1
      elif aux.iloc[t,1] == localteam:
        if aux.iloc[t,17] == "A":
          localstreak += 3
        elif aux.iloc[t,17] == "D":
          localstreak += 1
 
    rangeaway = 0
    awaystreak = 0
    aux = df[df.Home == awayteam]
    aux1 = df[df.Away == awayteam]
    aux = pd.concat([aux,aux1])
    aux = aux.sort_index()
    aux = aux.iloc[::-1]

    if len(aux) >= 5:
      rangeaway = 5
    else:
      rangeaway = len(aux)
    for t in range(rangeaway):
      if aux.iloc[t,0] == awayteam:
        if aux.iloc[t,17] == "H":
          awaystreak += 3
        elif aux.iloc[t,17] == "D":
          awaystreak += 1
      elif aux.iloc[t,1] == awayteam:
        if aux.iloc[t,17] == "A":
          awaystreak += 3
        elif aux.iloc[t,17] == "D":
          awaystreak += 1

    xgHlocal = 0 # xg del equipo local como  local
    xgAlocal = 0 #xg del equipo vistante como local
    xgHaway = 0 # xg del equipo local como  visitante
    xgAaway = 0 # xg del equipo visitante como  visitante
    avgxgHgoals = 0 #media xg totales equipo local
    avgxgAgoals = 0 #media xg totales equipo visitante

    if classification[indexlocalteam][4]!=0 or classification[indexlocalteam][2] !=0:
      xgHlocal = classification[indexlocalteam][4] / classification[indexlocalteam][2]
    if classification[indexawayteam][4]!=0 or classification[indexawayteam][2] !=0:
      xgAlocal = classification[indexawayteam][4] / classification[indexawayteam][2]
    if classification[indexlocalteam][6]!=0 or classification[indexlocalteam][3] !=0:
      xgHaway = classification[indexlocalteam][6] / classification[indexlocalteam][3]
    if classification[indexawayteam][6]!=0 or classification[indexawayteam][3] !=0:
      xgAaway = classification[indexawayteam][6] / classification[indexawayteam][3]

    avgxgHgoals = (xgHlocal + xgHaway)/2 
    avgxgAgoals = (xgAlocal + xgAaway)/2 

    xgHlocalconceded = 0 # xg encajados del equipo local como  local
    xgAlocalconceded = 0 # xg encajados del equipo vistante como local
    xgHawayconceded = 0 # xg encajados del equipo local como  visitante
    xgAawayconceded = 0 # xg encajados del equipo visitante como  visitante
    avgTotalHxgconceded = 0#media xg encajados totales equipo local
    avgTotalAxgconceded = 0#media xg encajados totales equipo visitante

    if classification[indexlocalteam][5]!=0 or classification[indexlocalteam][2] !=0:
      xgHlocalconceded = classification[indexlocalteam][5]/ classification[indexlocalteam][2]
    if classification[indexawayteam][5]!=0 or classification[indexawayteam][2] !=0:
      xgAlocalconceded = classification[indexawayteam][5] / classification[indexawayteam][2]
    if classification[indexlocalteam][7]!=0 or classification[indexlocalteam][3] !=0:
      xgHawayconceded = classification[indexlocalteam][7] / classification[indexlocalteam][3]
    if classification[indexawayteam][7]!=0 or classification[indexawayteam][3] !=0:
      xgAawayconceded = classification[indexawayteam][7] / classification[indexawayteam][3]

    avgTotalHxgconceded = (xgHlocalconceded + xgHawayconceded)/2 
    avgTotalAxgconceded = (xgAlocalconceded + xgAawayconceded)/2 

    especial = datacsv.loc[i][6] #partido especial
    mgh = datacsv.loc[i][7] #disponibilidad máximo goleador local
    mga = datacsv.loc[i][8] #disponibilidad máximo goleador visitante
    mah = datacsv.loc[i][9] #disponibilidad máximo asistente local
    maa = datacsv.loc[i][10] #disponibilidad máximo asistente visitante

    persaveH = 0 #porcentaje paradas portero local
    persaveA = 0 #porcentaje paradas portero visitante
    if classification[indexlocalteam][8]!=0 and (classification[indexlocalteam][2] !=0 or classification[indexlocalteam][3] !=0):
      persaveH = classification[indexlocalteam][8]/ (classification[indexlocalteam][2] + classification[indexlocalteam][3])
    if classification[indexawayteam][8]!=0 and (classification[indexawayteam][2] !=0 or classification[indexawayteam][3] !=0):
      persaveA = classification[indexawayteam][8] / (classification[indexlocalteam][2] + classification[indexlocalteam][3])

    result = ""
    if datacsv.loc[i][2] > datacsv.loc[i][3]:
      classification[indexlocalteam][1] += 3
      result = "H"
    elif datacsv.loc[i][2] == datacsv.loc[i][3]:
      classification[indexlocalteam][1] += 1
      classification[indexawayteam][1] += 1
      result = "D"
    else: 
      classification[indexawayteam][1] += 3
      result = "A"
    
    a = [localteam,awayteam,pointsH,pointsA,avgxgHgoals,avgxgAgoals,avgTotalHxgconceded,avgTotalAxgconceded,
         localstreak,awaystreak,especial,mgh,mga,mah,maa,persaveH,persaveA,result]

    df.loc[len(df)] = a

    classification[indexlocalteam][2] += 1
    classification[indexawayteam][3] += 1
    classification[indexlocalteam][4] += datacsv.loc[i][4]
    classification[indexlocalteam][5] += datacsv.loc[i][5]
    classification[indexawayteam][6] += datacsv.loc[i][5]
    classification[indexawayteam][7] += datacsv.loc[i][4]
    classification[indexlocalteam][8] += datacsv.loc[i][11]
    classification[indexawayteam][8] += datacsv.loc[i][12]

  return df, classification

Modelo basado en el sistema ELO

In [9]:
#función nA
def my_logisticA(xa, ha, da):
  return (3 / (1 + 10**(-(xa+ha)/da)))

#función nB
def my_logisticB(xb, hb, db):
  return (3 / (1 + 10**((xb+hb)/db)))

def elomodel(filename, partidosjugados):

  data = pd.read_csv(filename, sep=";")

  classification = []
  nA = []
  nB = []

  #Recoge el nombre de los equipos
  for i in range(10):
    classification.append([data.loc[i][0], 0, 0])
    classification.append([data.loc[i][1], 0, 0])

  #Recoge los puntos obtenidos según el marcador del partido y los valores de diferencia pA-pB y puntos obtenidos para la función sigmoide que se calculará
  for i in range(partidosjugados):
    tupleA = []
    tupleB = []
    A = data.loc[i][0]
    B = data.loc[i][1]
    pA = classification[getindexteam(A,classification)][1]
    pB = classification[getindexteam(B,classification)][1]
    tupleA.append(pA-pB)
    tupleB.append(pA-pB)
    if data.loc[i][2] > data.loc[i][3]:
      indexteam = getindexteam(data.loc[i][0],classification)
      classification[indexteam][1] += 3
      tupleA.append(3)
      tupleB.append(0)
    elif data.loc[i][2] == data.loc[i][3]:
      indexteam = getindexteam(data.loc[i][0],classification)
      classification[indexteam][1] += 1
      indexteam = getindexteam(data.loc[i][1],classification)
      classification[indexteam][1] += 1
      tupleA.append(1)
      tupleB.append(1)
    else: 
      indexteam = getindexteam(data.loc[i][1],classification)
      classification[indexteam][1] += 3
      tupleA.append(0)
      tupleB.append(3)
    nA.append(tupleA)
    nB.append(tupleB)
    

  dfa = pd.DataFrame(nA, columns =['x', 'y'])
  dfb = pd.DataFrame(nB, columns =['x', 'y'])

  #convierte las columnas del dataframe en arrays
  xa = np.array(dfa['x']) 
  ya = np.array(dfa['y'])
  xb = np.array(dfb['x']) 
  yb = np.array(dfb['y'])


  #Optimización de los parámetros para obtener los adecuados
  (ha,da),cova = optim.curve_fit(my_logisticA, xa, ya)

  (hb,db),covb = optim.curve_fit(my_logisticB, xb, yb)

  #Aplicación de las funciones a los partidos restantes que se deben predecir
  for i in range(partidosjugados,380):
    A = data.loc[i][0]
    B = data.loc[i][1]
    pA = classification[getindexteam(A,classification)][1]
    pB = classification[getindexteam(B,classification)][1]
    valorA = my_logisticA(pA-pB,ha,da)
    valorB = my_logisticB(pA-pB,hb,db)
    classification[getindexteam(A,classification)][2] += valorA 
    classification[getindexteam(B,classification)][2] += valorB 

  #Suma los puntos obtenidos de partidos jugados a los puntos predecidos
  for x in classification:
    x[2] = x[2] + x[1]

  return classification

In [10]:
#función que prepara las muestras para los partidos a predecir
def preparedata(matches1x1x, classification1x1x):
  features = matches1x1x[['Home','Away','pointsH','pointsA','xglocal','xgaway','xglocalconceded','xgawayconceded','localstreak','awaystreak','especial','persavH','persavA','Result']]
  x = features[['Home','Away','pointsH','pointsA','xglocal','xgaway','xglocalconceded','xgawayconceded','localstreak','awaystreak','especial','persavH','persavA']]
  y = features[['Result']]

  #datos entreamiento = 28 primeras jornadas, datos test = 10 últimas jornadas
  x_train = x.head(280)
  x_train = x_train[['pointsH','pointsA','xglocal','xgaway','xglocalconceded','xgawayconceded','localstreak','awaystreak','especial','persavH','persavA']]
  x_test = x.tail(100)
  x_test.reset_index(inplace = True) #resetea indices del df
  y_train = y.head(280)
  y_test = y.tail(100)

  #recogida de los datos justo antes de los partidos a empezar a predecir
  classificationdata = pd.DataFrame(columns =['team','points','avgxg','avgxgconceded','streak','persav'])
  futurematches = x_test[['Home','Away']]

  for i in range(10):
    a = [x.iloc[i][0],0,0,0,0,0]
    classificationdata.loc[len(classificationdata)] = a
    a = [x.iloc[i][1],0,0,0,0,0]
    classificationdata.loc[len(classificationdata)] = a

  listaequipos = []
  for i in range(20):
    if len(listaequipos) != 20:
      for j in range(20):
        if x_test.loc[i]['Home'] == classificationdata.loc[j]['team'] and x_test.loc[i]['Home'] not in listaequipos:
          classificationdata.loc[j]['points'] = x_test.loc[i]['pointsH']
          classificationdata.loc[j]['avgxg'] = x_test.loc[i]['xglocal']
          classificationdata.loc[j]['avgxgconceded'] = x_test.loc[i]['xglocalconceded']
          classificationdata.loc[j]['streak'] = x_test.loc[i]['localstreak']
          classificationdata.loc[j]['persav'] = x_test.loc[i]['persavH']
          listaequipos.append(x_test.loc[i]['Home'])
        if x_test.loc[i]['Away'] == classificationdata.loc[j]['team'] and x_test.loc[i]['Away'] not in listaequipos:
          classificationdata.loc[j]['points'] = x_test.loc[i]['pointsA']
          classificationdata.loc[j]['avgxg'] = x_test.loc[i]['xgaway']
          classificationdata.loc[j]['avgxgconceded'] = x_test.loc[i]['xgawayconceded']
          classificationdata.loc[j]['streak'] = x_test.loc[i]['awaystreak']
          classificationdata.loc[j]['persav'] = x_test.loc[i]['persavA']
          listaequipos.append(x_test.loc[i]['Away'])
    else:
      break

  #preparación de los datos x_test
  new_x_test = pd.DataFrame(columns =['pointsH','pointsA','xglocal','xgaway','xglocalconceded','xgawayconceded','localstreak','awaystreak','especial','persavH','persavA'])
  for i in range(100):
    home = x_test.loc[i]['Home']
    away = x_test.loc[i]['Away']
    homedf = classificationdata[classificationdata['team'] == home]
    awaydf = classificationdata[classificationdata['team'] == away]
    a = []
    a.append(homedf.iloc[0][1])
    a.append(awaydf.iloc[0][1])
    a.append(homedf.iloc[0][2])
    a.append(awaydf.iloc[0][2])
    a.append(homedf.iloc[0][3])
    a.append(awaydf.iloc[0][3])
    a.append(homedf.iloc[0][4])
    a.append(awaydf.iloc[0][4])
    a.append(x_test.loc[i]['especial'])
    a.append(homedf.iloc[0][5])
    a.append(awaydf.iloc[0][5])

    new_x_test.loc[len(new_x_test)] = a
  
  return classificationdata, x_train, y_train, new_x_test, y_test, futurematches


# Experimento 4

Proceso A

In [18]:
#Función que dado los resultados de los modelos calcula la clasificación final
def calculateclassification(futurematches,y_pred, classificationdata):
  cteam = []
  cpoints = []
  for i in range(20):
    a = []
    a.append(classificationdata.iloc[i,0])
    cteam.append(a)
    cpoints.append(classificationdata.iloc[i,1])
  
  for i in range(100):
    home = futurematches.loc[i]['Home']
    away = futurematches.loc[i]['Away']
    if y_pred[i] == 'H':
      index = getindexteam(home,cteam)
      cpoints[index] += 3
    elif y_pred[i] == 'D':
      index = getindexteam(home,cteam)
      cpoints[index] += 1
      index = getindexteam(away,cteam)
      cpoints[index] += 1
    elif y_pred[i] == 'A':
      index = getindexteam(away,cteam)
      cpoints[index] += 3
    
  return cpoints


def modelsclassification(classificationdata, classification1x1x, eloclassification, x_train, y_train, new_x_test, y_test, futurematches):

  predictedclassification = pd.DataFrame(columns =['model',classificationdata.loc[0][0], classificationdata.loc[1][0], classificationdata.loc[2][0], classificationdata.loc[3][0],
                                                  classificationdata.loc[4][0], classificationdata.loc[5][0], classificationdata.loc[6][0], classificationdata.loc[7][0], classificationdata.loc[8][0],
                                                  classificationdata.loc[9][0],classificationdata.loc[10][0],classificationdata.loc[11][0],classificationdata.loc[12][0],classificationdata.loc[13][0],
                                                  classificationdata.loc[14][0],classificationdata.loc[15][0],classificationdata.loc[16][0],classificationdata.loc[17][0],classificationdata.loc[18][0],
                                                  classificationdata.loc[19][0]])


  pc=[]
  for i in range(20):
    for j in range (20):
      if classificationdata.loc[i][0] == classification1x1x[j][0]:
        pc.append(classification1x1x[j][1])
        break
      
  pc.insert(0, "Real classification")
  predictedclassification.loc[len(predictedclassification)] = pc

  pc=[]
  for i in range(20):
    for j in range (20):
      if classificationdata.loc[i][0] == eloclassification[j][0]:
        pc.append(round(eloclassification[j][2], 2))
        break
  pc.insert(0, "ELO")
  predictedclassification.loc[len(predictedclassification)] = pc

  logreg = LogisticRegression()
  logreg.fit(x_train, y_train.values.ravel())
  y_pred_LR = logreg.predict(new_x_test)
  pc = calculateclassification(futurematches,y_pred_LR, classificationdata)
  pc.insert(0, "Logistic Regression")
  predictedclassification.loc[len(predictedclassification)] = pc

  linearsvm = SVC(kernel='linear')
  linearsvm.fit(x_train, y_train.values.ravel())
  y_pred_LSVM = linearsvm.predict(new_x_test)
  pc = calculateclassification(futurematches,y_pred_LSVM, classificationdata)
  pc.insert(0, "Linear kernel SVM")
  predictedclassification.loc[len(predictedclassification)] = pc

  gnb = GaussianNB()
  gnb.fit(x_train, y_train.values.ravel())
  y_pred_GNB = gnb.predict(new_x_test)
  pc = calculateclassification(futurematches,y_pred_GNB, classificationdata)
  pc.insert(0, "Gaussian Naive Bayes")
  predictedclassification.loc[len(predictedclassification)] = pc


  errorpointclassification = []
  for i in range(len(predictedclassification)-1):
    dif = 0
    for j in range(20):
      dif += abs(predictedclassification.iloc[0,j+1] - predictedclassification.iloc[i+1,j+1])
    dif = round(dif/20,2)
    errorpointclassification.append(dif)

  return errorpointclassification, predictedclassification

In [ ]:
errorresults = pd.DataFrame({'model': ["Guyon","Logistic Regression", "Linear kernel SVM", "Gaussian Naive Bayes"], 
                            'avg 1718': [0,0,0,0],
                            'avg 1920': [0,0,0,0],
                            'avg total': [0,0,0,0],
                            })

pclasif1718 = pd.DataFrame()
pclasif1819 = pd.DataFrame()

matches1718, classification1718 = getdatafromcsv("LaLigaData1718.csv")
classificationdata1718, x_train1718, y_train1718, new_x_test1718, y_test1718, futurematches1718 = preparedata(matches1718,classification1718) 
eloclassification1718 = elomodel("LaLigaData1718.csv", 280)

matches1819, classification1819 = getdatafromcsv("LaLigaData1819.csv")
classificationdata1819, x_train1819, y_train1819, new_x_test1819, y_test1819, futurematches1819 = preparedata(matches1819,classification1819) 
eloclassification1819 = elomodel("LaLigaData1819.csv", 280)

iteraciones = 1
for i in range(iteraciones):
  print(i)

  errorpoints1718,predictedclassification1718 = modelsclassification(classificationdata1718, classification1718, eloclassification1718,
                                                                                 x_train1718, y_train1718, new_x_test1718, y_test1718, futurematches1718)
  
  errorpoints1819,predictedclassification1819 = modelsclassification(classificationdata1819, classification1819, eloclassification1819,
                                                                                 x_train1819, y_train1819, new_x_test1819, y_test1819, futurematches1819)

  
  for t in range(4):
    errorresults.iloc[t,1] += errorpoints1718[t]
    errorresults.iloc[t,2] += errorpoints1819[t]

  if i ==0:
    pclasif1718 = predictedclassification1718
    pclasif1819 = predictedclassification1819
  else:
    for y in range(2,5):
      for w in range(1,21):
        pclasif1718.iloc[y,w] += predictedclassification1718.iloc[y,w]
        pclasif1819.iloc[y,w] += predictedclassification1819.iloc[y,w]


for t in range(4):
  errorresults.iloc[t,1] = errorresults.iloc[t,1] / iteraciones
  errorresults.iloc[t,2] = errorresults.iloc[t,2] / iteraciones
  errorresults.iloc[t,3] = (errorresults.iloc[t,1] + errorresults.iloc[t,2])/(2)


for y in range(2,5):
  for w in range(1,21):
    pclasif1718.iloc[y,w] = pclasif1718.iloc[y,w] / iteraciones
    pclasif1819.iloc[y,w] = pclasif1819.iloc[y,w] / iteraciones

print("\n")
print("Media del error de puntos")
display(errorresults.head(10))
print("\n")
print("Clasificación temporada 17-18")
display(pclasif1718.head(10))
print("\n")
print("Clasificación temporada 18-19")
display(pclasif1819.head(10))



Proceso B

In [22]:
#función que dado los resultados de los modelos calcula la clasificación final
def calculateclassificationprobs(futurematches,y_pred_probs, classificationdata):

  cteam = []
  cpoints = []
  for i in range(20):
    a = []
    a.append(classificationdata.iloc[i,0])
    cteam.append(a)
    cpoints.append(classificationdata.iloc[i,1])
  
  for i in range(100):
    home = futurematches.loc[i]['Home']
    away = futurematches.loc[i]['Away']

    probhome = y_pred_probs[i][2]
    probdraw = y_pred_probs[i][1]
    probaway = y_pred_probs[i][0]
    p = np.array([probhome,probdraw,probaway])
    p /= p.sum()
    res = np.random.choice(["H", "D", "A"],p=p)

    if res == 'H':
      index = getindexteam(home,cteam)
      cpoints[index] += 3
    elif res == 'D':
      index = getindexteam(home,cteam)
      cpoints[index] += 1
      index = getindexteam(away,cteam)
      cpoints[index] += 1
    elif res == 'A':
      index = getindexteam(away,cteam)
      cpoints[index] += 3

  return cpoints


def modelsclassificationprobs(classificationdata, classification1x1x, eloclassification, x_train, y_train, new_x_test, y_test, futurematches):

  predictedclassification = pd.DataFrame(columns =['model',classificationdata.loc[0][0], classificationdata.loc[1][0], classificationdata.loc[2][0], classificationdata.loc[3][0],
                                                  classificationdata.loc[4][0], classificationdata.loc[5][0], classificationdata.loc[6][0], classificationdata.loc[7][0], classificationdata.loc[8][0],
                                                  classificationdata.loc[9][0],classificationdata.loc[10][0],classificationdata.loc[11][0],classificationdata.loc[12][0],classificationdata.loc[13][0],
                                                  classificationdata.loc[14][0],classificationdata.loc[15][0],classificationdata.loc[16][0],classificationdata.loc[17][0],classificationdata.loc[18][0],
                                                  classificationdata.loc[19][0]])


  pc=[]
  for i in range(20):
    for j in range (20):
      if classificationdata.loc[i][0] == classification1x1x[j][0]:
        pc.append(classification1x1x[j][1])
        break
      
  pc.insert(0, "Real classification")
  predictedclassification.loc[len(predictedclassification)] = pc

  pc=[]
  for i in range(20):
    for j in range (20):
      if classificationdata.loc[i][0] == eloclassification[j][0]:
        pc.append(round(eloclassification[j][2], 2))
        break
  pc.insert(0, "ELO")
  predictedclassification.loc[len(predictedclassification)] = pc
  
  logreg = LogisticRegression()
  logreg.fit(x_train, y_train.values.ravel())
  y_predict_LR_probs = logreg.predict_proba(new_x_test)
  pc = calculateclassificationprobs(futurematches,y_predict_LR_probs, classificationdata)
  pc.insert(0, "Logistic Regression")
  predictedclassification.loc[len(predictedclassification)] = pc
  
  linearsvm = SVC(kernel='linear', probability=True)
  linearsvm.fit(x_train, y_train.values.ravel())
  y_pred_LSVM_probs = linearsvm.predict_proba(new_x_test)
  pc = calculateclassificationprobs(futurematches,y_pred_LSVM_probs, classificationdata)
  pc.insert(0, "Linear kernel SVM")
  predictedclassification.loc[len(predictedclassification)] = pc

  gnb = GaussianNB()
  gnb.fit(x_train, y_train.values.ravel())
  y_pred_GNB_probs = gnb.predict_proba(new_x_test)
  pc = calculateclassificationprobs(futurematches,y_pred_GNB_probs, classificationdata)
  pc.insert(0, "Gaussian Naive Bayes")
  predictedclassification.loc[len(predictedclassification)] = pc

  errorpointclassification = []
  for i in range(len(predictedclassification)-1):
    dif = 0
    for j in range(20):
      dif += abs(predictedclassification.iloc[0,j+1] - predictedclassification.iloc[i+1,j+1])
    dif = round(dif/20,2)
    errorpointclassification.append(dif)

  return errorpointclassification, predictedclassification

In [ ]:
errorresults = pd.DataFrame({'model': ["Guyon","Logistic Regression", "Linear kernel SVM", "Gaussian Naive Bayes"], 
                            'avg 1718': [0,0,0,0],
                            'avg 1920': [0,0,0,0],
                            'avg total': [0,0,0,0],
                            })

pclasif1718 = pd.DataFrame()
pclasif1819 = pd.DataFrame()

matches1718, classification1718 = getdatafromcsv("LaLigaData1718.csv")
classificationdata1718, x_train1718, y_train1718, new_x_test1718, y_test1718, futurematches1718 = preparedata(matches1718,classification1718) 
eloclassification1718 = elomodel("LaLigaData1718.csv", 280)

matches1819, classification1819 = getdatafromcsv("LaLigaData1819.csv")
classificationdata1819, x_train1819, y_train1819, new_x_test1819, y_test1819, futurematches1819 = preparedata(matches1819,classification1819) 
eloclassification1819 = elomodel("LaLigaData1819.csv", 280)


iteraciones = 2
for i in range(iteraciones):
  print(i)

  errorpoints1718,predictedclassification1718 = modelsclassificationprobs(classificationdata1718, classification1718, eloclassification1718,
                                                                                 x_train1718, y_train1718, new_x_test1718, y_test1718, futurematches1718)
  
  errorpoints1819,predictedclassification1819 = modelsclassificationprobs(classificationdata1819, classification1819, eloclassification1819,
                                                                                 x_train1819, y_train1819, new_x_test1819, y_test1819, futurematches1819)

  
  for t in range(4):
    errorresults.iloc[t,1] += errorpoints1718[t]
    errorresults.iloc[t,2] += errorpoints1819[t]

  if i ==0:
    pclasif1718 = predictedclassification1718
    pclasif1819 = predictedclassification1819
  else:
    for y in range(2,5):
      for w in range(1,21):
        pclasif1718.iloc[y,w] += predictedclassification1718.iloc[y,w]
        pclasif1819.iloc[y,w] += predictedclassification1819.iloc[y,w]


for t in range(4):
  errorresults.iloc[t,1] = errorresults.iloc[t,1] / iteraciones
  errorresults.iloc[t,2] = errorresults.iloc[t,2] / iteraciones
  errorresults.iloc[t,3] = (errorresults.iloc[t,1] + errorresults.iloc[t,2])/(2)

for y in range(2,5):
  for w in range(1,21):
    pclasif1718.iloc[y,w] = pclasif1718.iloc[y,w] / iteraciones
    pclasif1819.iloc[y,w] = pclasif1819.iloc[y,w] / iteraciones

print("\n")
print("Media del error de puntos")
display(errorresults.head(10))
print("\n")
print("Clasificación temporada 17-18")
display(pclasif1718.head(10))
print("\n")
print("Clasificación temporada 18-19")
display(pclasif1819.head(10))


Proceso C

In [41]:

def preparenewdata(classificationdata, futurematches, jornada, df):
  
  #preparación de los datos x_test
  new_x_test = pd.DataFrame(columns =['pointsH','pointsA','xglocal','xgaway','xglocalconceded','xgawayconceded','localstreak','awaystreak','especial','persavH','persavA'])
  for i in range(jornada*10,jornada*10+10):
    home = futurematches.loc[i]['Home']
    away = futurematches.loc[i]['Away']
    homedf = classificationdata[classificationdata['team'] == home]
    awaydf = classificationdata[classificationdata['team'] == away]
    a = []
    a.append(homedf.iloc[0][1])
    a.append(awaydf.iloc[0][1])
    a.append(homedf.iloc[0][2])
    a.append(awaydf.iloc[0][2])
    a.append(homedf.iloc[0][3])
    a.append(awaydf.iloc[0][3])
    a.append(homedf.iloc[0][4])
    a.append(awaydf.iloc[0][4])
    a.append(df.iloc[280+i][10])
    a.append(homedf.iloc[0][5])
    a.append(awaydf.iloc[0][5])

    new_x_test.loc[len(new_x_test)] = a
  
  return new_x_test


In [42]:
#función que dado los resultados de los modelos calcula la clasificación final
def calculateclassificationprobs(futurematches,y_pred_probs, classificationdata,jornada, df):
  
  for i in range(0,10):
    home = futurematches.loc[i+jornada*10]['Home']
    away = futurematches.loc[i+jornada*10]['Away']

    probhome = y_pred_probs[i][2]
    probdraw = y_pred_probs[i][1]
    probaway = y_pred_probs[i][0]

    res = np.random.choice(["H", "D", "A"],p=[probhome,probdraw,probaway])

    a=[home,away,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    
    if res == 'H':
      classificationdata.loc[classificationdata['team'] == home, 'points'] += 3
      a.append("H")
    elif res == 'D':
      classificationdata.loc[classificationdata['team'] == home, 'points'] += 1
      classificationdata.loc[classificationdata['team'] == away, 'points'] += 1
      a.append("D")
    elif res == 'A':
      classificationdata.loc[classificationdata['team'] == away, 'points'] += 3
      a.append("A")

    df.loc[len(df)] = a

    rangelocal = 0 #racha local de los últimos cinco partidos
    localstreak = 0
    aux = df[df.Home == home]
    aux1 = df[df.Away == home]
    aux = pd.concat([aux,aux1])
    aux = aux.sort_index()
    aux = aux.iloc[::-1]
    
    if len(aux) >= 5:
      rangelocal = 5
    else:
      rangelocal = len(aux)
    for t in range(rangelocal):
      if aux.iloc[t,0] == home:
        if aux.iloc[t,17] == "H":
          localstreak += 3
        elif aux.iloc[t,17] == "D":
          localstreak += 1
      elif aux.iloc[t,1] == home:
        if aux.iloc[t,17] == "A":
          localstreak += 3
        elif aux.iloc[t,17] == "D":
          localstreak += 1

    rangeaway = 0 #racha visitante de los últimos cinco partidos
    awaystreak = 0
    aux = df[df.Home == away]
    aux1 = df[df.Away == away]
    aux = pd.concat([aux,aux1])
    aux = aux.sort_index()
    aux = aux.iloc[::-1]
    if len(aux) >= 5:
      rangeaway = 5
    else:
      rangeaway = len(aux)
    for t in range(rangeaway):
      if aux.iloc[t,0] == away:
        if aux.iloc[t,17] == "H":
          awaystreak += 3
        elif aux.iloc[t,17] == "D":
          awaystreak += 1
      elif aux.iloc[t,1] == away:
        if aux.iloc[t,17] == "A":
          awaystreak += 3
        elif aux.iloc[t,17] == "D":
          awaystreak += 1

    classificationdata.loc[classificationdata['team'] == home, 'streak'] = localstreak
    classificationdata.loc[classificationdata['team'] == away, 'streak'] = awaystreak

  return classificationdata, df


def modelsclassificationprobs(classificationdata, classification1x1x, eloclassification, x_train, y_train, new_x_test, y_test, futurematches, df):

  predictedclassification = pd.DataFrame(columns =['model',classificationdata.loc[0][0], classificationdata.loc[1][0], classificationdata.loc[2][0], classificationdata.loc[3][0],
                                                  classificationdata.loc[4][0], classificationdata.loc[5][0], classificationdata.loc[6][0], classificationdata.loc[7][0], classificationdata.loc[8][0],
                                                  classificationdata.loc[9][0],classificationdata.loc[10][0],classificationdata.loc[11][0],classificationdata.loc[12][0],classificationdata.loc[13][0],
                                                  classificationdata.loc[14][0],classificationdata.loc[15][0],classificationdata.loc[16][0],classificationdata.loc[17][0],classificationdata.loc[18][0],
                                                  classificationdata.loc[19][0]])

  pc=[]
  for i in range(20):
    for j in range (20):
      if classificationdata.loc[i][0] == classification1x1x[j][0]:
        pc.append(classification1x1x[j][1])
        break
  pc.insert(0, "Real classification")
  predictedclassification.loc[len(predictedclassification)] = pc

  pc=[]
  for i in range(20):
    for j in range (20):
      if classificationdata.loc[i][0] == eloclassification[j][0]:
        pc.append(round(eloclassification[j][2], 2))
        break
  pc.insert(0, "ELO")
  predictedclassification.loc[len(predictedclassification)] = pc

  classificationdata1 = classificationdata.copy()
  df1 = df[0:280].copy()
  classificationdata2 = classificationdata.copy()
  df2 = df[0:280].copy()
  classificationdata5 = classificationdata.copy()
  df5 = df[0:280].copy()

  logreg = LogisticRegression()
  logreg.fit(x_train, y_train.values.ravel())
  linearsvm = SVC(kernel='linear', probability=True)
  linearsvm.fit(x_train, y_train.values.ravel())
  gnb = GaussianNB()
  gnb.fit(x_train, y_train.values.ravel())

  for i in range(10):

    new_x_test = preparenewdata(classificationdata1, futurematches, i, df)
    y_predict_LR_probs = logreg.predict_proba(new_x_test)
    classif,dfc = calculateclassificationprobs(futurematches,y_predict_LR_probs, classificationdata1, i, df1)
    classificationdata1 = classif
    df1 = dfc

    new_x_test = preparenewdata(classificationdata2, futurematches, i, df)
    y_pred_LSVM_probs = linearsvm.predict_proba(new_x_test)
    classif,dfc = calculateclassificationprobs(futurematches,y_pred_LSVM_probs, classificationdata2, i, df2)
    classificationdata2 = classif
    df2 = dfc

    new_x_test = preparenewdata(classificationdata5, futurematches, i, df)
    y_pred_GNB_probs = gnb.predict_proba(new_x_test)
    classif,dfc = calculateclassificationprobs(futurematches,y_pred_GNB_probs, classificationdata5, i, df5)
    classificationdata5 = classif
    df5 = dfc


  a=[]
  a.append("Regresión logística")
  for j in range(20):
    a.append(classificationdata1.iloc[j][1])
  predictedclassification.loc[len(predictedclassification)] = a
  a=[]
  a.append("SVM lineal")
  for j in range(20):
    a.append(classificationdata2.iloc[j][1])
  predictedclassification.loc[len(predictedclassification)] = a
  a=[]
  a.append("Gaussian Naive Bayes")
  for j in range(20):
    a.append(classificationdata5.iloc[j][1])
  predictedclassification.loc[len(predictedclassification)] = a

  errorpointclassification = []
  for i in range(len(predictedclassification)-1):
    dif = 0
    for j in range(20):
      dif += abs(predictedclassification.iloc[0,j+1] - predictedclassification.iloc[i+1,j+1])
    dif = round(dif/20,2)
    errorpointclassification.append(dif)

  return errorpointclassification, predictedclassification

In [ ]:
errorresults = pd.DataFrame({'model': ["Guyon","Logistic Regression", "Linear kernel SVM", "Gaussian Naive Bayes"], 
                            'avg 1718': [0,0,0,0],
                            'avg 1920': [0,0,0,0],
                            'avg total': [0,0,0,0],
                            })

pclasif1718 = pd.DataFrame()
pclasif1819 = pd.DataFrame()

dfmatches1718, classification1718 = getdatafromcsv("LaLigaData1718.csv")
classificationdata1718, x_train1718, y_train1718, new_x_test1718, y_test1718, futurematches1718 = preparedata(matches1718,classification1718) 
eloclassification1718 = elomodel("LaLigaData1718.csv", 280)

dfmatches1819, classification1819 = getdatafromcsv("LaLigaData1819.csv")
classificationdata1819, x_train1819, y_train1819, new_x_test1819, y_test1819, futurematches1819 = preparedata(matches1819,classification1819) 
eloclassification1819 = elomodel("LaLigaData1819.csv", 280)

iteraciones = 2
for i in range(iteraciones):
  print(i)


  errorpoints1718,predictedclassification1718 = modelsclassificationprobs(classificationdata1718, classification1718, eloclassification1718,
                                                                                 x_train1718, y_train1718, new_x_test1718, y_test1718, futurematches1718, 
                                                                                 dfmatches1718)
  
  errorpoints1819,predictedclassification1819 = modelsclassificationprobs(classificationdata1819, classification1819, eloclassification1819,
                                                                                 x_train1819, y_train1819, new_x_test1819, y_test1819, futurematches1819,
                                                                                 dfmatches1819)


  for t in range(4):
    errorresults.iloc[t,1] += errorpoints1718[t]
    errorresults.iloc[t,2] += errorpoints1819[t]

  if i ==0:
    pclasif1718 = predictedclassification1718
    pclasif1819 = predictedclassification1819
  else:
    for y in range(2,5):
      for w in range(1,21):
        pclasif1718.iloc[y,w] += predictedclassification1718.iloc[y,w]
        pclasif1819.iloc[y,w] += predictedclassification1819.iloc[y,w]


for t in range(4):
  errorresults.iloc[t,1] = errorresults.iloc[t,1] / iteraciones
  errorresults.iloc[t,2] = errorresults.iloc[t,2] / iteraciones
  errorresults.iloc[t,3] = (errorresults.iloc[t,1] + errorresults.iloc[t,2])/(2)

for y in range(2,5):
  for w in range(1,21):
    pclasif1718.iloc[y,w] = pclasif1718.iloc[y,w] / iteraciones
    pclasif1819.iloc[y,w] = pclasif1819.iloc[y,w] / iteraciones

print("\n")
print("Media del error de puntos")
display(errorresults.head(10))
print("\n")
print("Clasificación temporada 17-18")
display(pclasif1718.head(10))
print("\n")
print("Clasificación temporada 18-19")
display(pclasif1819.head(10))
